In [2]:
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)
from sklearn import tree
import numpy as np
import pandas as pd 

# [Decision trees](https://en.wikipedia.org/wiki/Decision_tree_learning)

### What is it?
It is a non-parametric classification / regression model.

### When do I use it?
I want to predict / classify a response variable based on arbitrary data. Be it countinous (regression problem) or categorical (classification problem) response.

### Why do I use it?
Its super-simple to setup. Its super-simple to explain to a customer.

### Why do I NOT use it?
Its easily over-trained (i.e. not robust). It is unstable regarding the data (small change in input data leeds to completely different model).

### Why do we even talk about it?
It is a good building brick to build realy good models from (e.g. random forests ,gradient boosting).

# How does it work?

We have a data-set of customers with some features and response variable Y = age. We want to be able to predict the age in order to offer them student / senior discount. And we would like to setup a set of questions which will lead us to the approximate age of a person.

Something like: 1] Does he like games? -> YES. 2] Does he also like gardening? -> No. Then he is about 14!


**Note:** The questions are called internal nodes. The final nodes are called leafs.


Our data are:

browsed cell phones | browsed video games | browsed gardening | Age
------------- | ------------- | ------------- | -------------
Yes  | Yes | No | 13
Yes  | Yes | No | 11
Yes  | No | No | 27
Yes  | Yes | Yes | 43
No   | No | Yes | 68
Yes  | No | Yes | 72
No  | No | Yes | 57
yes  | Yes | No | 16

Now we need to find out what feture to ask for first. That should be one, that splits our data the most. That we do through minimizing impurity. 

## Math background

Impurity measures how badly is the data at node $Q$ separated by given feture and break point $\theta$. It is defined as:

$G(Q, \theta) = \frac{n_{left}}{N_m} H(Q_{left}(\theta)) + \frac{n_{right}}{N_m} H(Q_{right}(\theta))$

Where $Q_{left}$ represents the left side of the node. The inpurity function $H()$ can be defined in more ways. We will take the Gini inpurity which is defined as:

$H(Q_i) = \sum_{k} p_{ik}(1-p_{ik})$, where
$p_{ik} = \frac{\sum_{x_j \in Q_i}I(y_j = k)}{N_i}$,

where k = possible values of response variable, i = index of left or right node split. Q_i represents data in that split. x_j represents predictor values. y_j represents response values. N_i = total number of data points in Q_i. I(x) = identity function. 

**Note:** Other inpurity functions could be Cross-Entropy or Misclassification for classification problems. Or simply Mean Squared Error or Mean Absolute Error for regression problems.


We then choose the feature and breakpoint $\theta^*$ which has $\theta^* = argmin_{\theta} G(Q, \theta)$.


This is repeated until there is only single data point in every leaf.

### Overfitting
The overfitting is handeled by setting parameter of minimum datapoints in leaf.

## Sources
[Scikit documentation](http://scikit-learn.org/stable/modules/tree.html)

In [3]:
df = pd.DataFrame({'cell':[1,1,1,1,0,1,0,1],'games':[1,1,0,1,0,0,0,1],'garden':[0,0,0,1,1,1,1,0],'age':['junior','junior','mid-age','mid-age','senior','senior','senior','junior']})
df

,cell,games,garden,age
0,1,1,0,junior
1,1,1,0,junior
2,1,0,0,mid-age
3,1,1,1,mid-age
4,0,0,1,senior
5,1,0,1,senior
6,0,0,1,senior
7,1,1,0,junior


In [4]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(df[['cell','games','garden']], df[['age']])
import graphviz 
dot_data = tree.export_graphviz(clf, out_file=None, feature_names=['cell','games','garden'], class_names = ['junior','mid-age','senior']) 
graph = graphviz.Source(dot_data) 
graph.render("customer_age")

'customer_age.pdf'

<img src="./customer_age.pdf" />

In [5]:
# REGRESSION PROBLEM
# create X data for the regression
X = np.arange(0, 10, 0.1)
# add some salt to simulate noise
salt0 = [6*(np.random.rand() > .4)*(-1)**(np.random.rand() > .5) for x in X]
salt1 = [30*(np.random.rand() > .9)*(-1)**(np.random.rand() > .5) for x in X]
salt2 = [75*(np.random.rand() > .95)*(-1)**(np.random.rand() > .5) for x in X]
salt_outlier = [200*(np.random.rand() > .98)*(-1)**(np.random.rand() > .5) for x in X]
Y_pure = [50*np.sin(x)+x*15 for x in X]
# create Y data for the regression
Y = np.sum([Y_pure, salt0, salt1, salt2, salt_outlier], axis = 0)

In [6]:
# plot the [X, Y] data
iplot([go.Scatter(x=X, y=Y, mode = 'markers')])

In [13]:
# set the parameter for minimal number of data points in leaf nodes
n_leafs = 4

# initialize the model object instance
clf = tree.DecisionTreeRegressor(min_samples_leaf = n_leafs)
# fit the model
clf = clf.fit(X.reshape(-1,1), Y)
# make the prediction for the data on which the model was fitted
Y_hat = clf.predict(X.reshape(-1,1))

# plot the data & fitted values with plotly
trace0 = go.Scatter(x=X, y=Y, mode = 'markers', name = 'Original data')
trace1 = go.Scatter(x=X, y=Y_hat, name = 'Prediction')
data = [trace0, trace1]
iplot(data)

In [201]:
clf0 = tree.DecisionTreeRegressor(min_samples_leaf = 0.04)
clf1 = tree.DecisionTreeRegressor(min_samples_leaf = 0.04, criterion='mae')
clf0_fit = clf0.fit(X.reshape(-1,1), Y)
clf1_fit = clf1.fit(X.reshape(-1,1), Y)

Y_hat0 = clf0_fit.predict(X.reshape(-1,1))
Y_hat1 = clf1_fit.predict(X.reshape(-1,1))

trace0 = go.Scatter(x=X, y=Y, mode = 'markers', name = 'Original data')
trace1 = go.Scatter(x=X, y=Y_hat0, name = 'Mean Squared Error criterion')
trace2 = go.Scatter(x=X, y=Y_hat1, name = 'Mean Absolute Error criterion')

data = [trace0, trace1, trace2]
iplot(data)